In [10]:
import requests
import json
import forecastio
from pprint import pprint
from geopy.geocoders import Nominatim
import pandas as pd
import import_ipynb
from WashingtonDC_airbnb import WashingtonDC_airbnb
import datetime as dt

importing Jupyter notebook from WashingtonDC_airbnb.ipynb


WashingtonDC_airbnb.ipynb:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  {
WashingtonDC_airbnb.ipynb:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  "cells": [
WashingtonDC_airbnb.ipynb:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  "outputs": [],


In [2]:
api_key = "a36c8345cc74c469a50078545e4970de"

In [3]:
geolocator = Nominatim(user_agent="ETL_Project")

location = geolocator.geocode("Washington DC")

DC_coord = []
DC_coord.append(location.latitude)
DC_coord.append(location.longitude)

DC_coord

[38.8950092, -77.0365625]

In [5]:
forecast = forecastio.load_forecast(api_key, DC_coord[0], DC_coord[1])

byDay = forecast.daily()
print(byDay.summary)
print(byDay.icon)
print(byDay.data[4].temperatureHigh)

Light rain today through Friday, with high temperatures bottoming out at 42°F on Monday.
rain
50.06


In [6]:
temp_high = []
temp_low = []
precip_perc = []
precip_type = []
precip_int = []
date = []

for i in byDay.data:
    date.append(i.time.date())
    temp_high.append(i.temperatureHigh)
    temp_low.append(i.temperatureLow)
    precip_perc.append(i.precipProbability)
    precip_int.append(i.precipIntensity)
    try:
        precip_type.append(i.precipType)
    except:
        precip_type.append("none")

date

dates = [dt.datetime.strftime(i, '%Y-%m-%d') for i in date]
dates

['2018-12-22',
 '2018-12-23',
 '2018-12-24',
 '2018-12-25',
 '2018-12-26',
 '2018-12-27',
 '2018-12-28',
 '2018-12-29']

In [7]:
df = pd.DataFrame({"Date": dates, 
                     "High Temp": temp_high, 
                     "Low Temp": temp_low, 
                     "Precipitation Probability": precip_perc,
                     "Precipitation Intensity": precip_int,
                     "Precipitation Type": precip_type
                    })
df.dtypes

Date                          object
High Temp                    float64
Low Temp                     float64
Precipitation Probability    float64
Precipitation Intensity      float64
Precipitation Type            object
dtype: object

In [11]:
DC_table = pd.merge(df, WashingtonDC_airbnb, on="Date")
DC_table

,Date,High Temp,Low Temp,Precipitation Probability,Precipitation Intensity,Precipitation Type,Under $50,$50-100,$100-150,$150-200,$200-250
0,2018-12-22,48.79,31.65,0.00,0.0000,none,472,1559,923,543,267
1,2018-12-23,46.68,37.00,0.38,0.0019,rain,485,1542,953,495,263
2,2018-12-24,41.59,33.21,0.29,0.0007,rain,470,1498,940,487,253
3,2018-12-25,44.49,30.42,0.00,0.0000,none,468,1461,921,494,241
4,2018-12-26,50.06,33.56,0.00,0.0000,none,470,1433,910,482,244
5,2018-12-27,42.69,44.17,0.68,0.0115,rain,473,1420,904,470,238
6,2018-12-28,64.72,46.05,0.96,0.0291,rain,451,1375,876,475,247
